In [10]:
import survivalstan
import numpy as np
from stancache import stancache
import pandas as pd

Load the `lung` dataset from the `survival` package in R

In [11]:
import statsmodels.api as sm
lung = sm.datasets.get_rdataset("lung", "survival")

<class 'statsmodels.datasets.utils.Dataset'>

In [7]:
print(lung.__doc__)

+--------+-------------------+
| lung   | R Documentation   |
+--------+-------------------+

NCCTG Lung Cancer Data
----------------------

Description
~~~~~~~~~~~

Survival in patients with advanced lung cancer from the North Central
Cancer Treatment Group. Performance scores rate how well the patient can
perform usual daily activities.

Usage
~~~~~

::

    lung
    cancer

Format
~~~~~~

inst:

Institution code

time:

Survival time in days

status:

censoring status 1=censored, 2=dead

age:

Age in years

sex:

Male=1 Female=2

ph.ecog:

ECOG performance score (0=good 5=dead)

ph.karno:

Karnofsky performance score (bad=0-good=100) rated by physician

pat.karno:

Karnofsky performance score as rated by patient

meal.cal:

Calories consumed at meals

wt.loss:

Weight loss in last six months

Source
~~~~~~

Terry Therneau

References
~~~~~~~~~~

Loprinzi CL. Laurie JA. Wieand HS. Krook JE. Novotny PJ. Kugler JW.
Bartel J. Law M. Bateman M. Klatt NE. et al. Prospective evaluation of


In [29]:
import numpy as np
lung.data['event'] = lung.data['status']==2 
lung.data['age_centered'] = lung.data['age'] - np.mean(lung.data['age'])
lung.data['male'] = lung.data['sex'] == 1
lung.data['ecog'] = lung.data['ph.ecog'].astype('category').values
lung.data.reset_index(inplace=True)
dflong = survivalstan.prep_data_long_surv(df=lung.data, event_col='event', time_col='time')


In [ ]:
fit1 = survivalstan.fit_stan_survival_model(
    df=dflong,
    model_code=survivalstan.models.pem_survival_model_timevarying,
    sample_col = 'index',
    timepoint_end_col = 'end_time',
    event_col = 'end_failure',
    formula = '~ age_centered + male + ecog',
    iter = 5000,
    chains = 4,
    model_cohort = 'time-dep: age + age + ecog',
    FIT_FUN = stancache.cached_stan_fit, 
)

INFO:stancache.stancache:Step 1: Get compiled model code, possibly from cache
INFO:stancache.stancache:StanModel: cache_filename set to anon_model.cython_0_25_2.model_code_9304163442804524267.pystan_2_12_0_0.stanmodel.pkl
INFO:stancache.stancache:StanModel: Starting execution
INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_83e0ce39a3e85f01b32b9c74bec9aa93 NOW.
